In [1]:
# using Pkg; Pkg.add("DataFrames"); Pkg.add("CSV")

In [2]:
download("https://openmv.net/file/class-grades.csv", "grades.csv")


"grades.csv"

In [3]:
using CSV, DataFrames, Statistics

In [4]:
df = CSV.read("grades.csv");

In [5]:
summary(df)

"99×6 DataFrame"

In [6]:
describe(df, :min, :max, :nmissing)

,variable,min,max,nmissing
,Symbol,Real,Real,Union…
1,Prefix,4,8,
2,Assignment,28.14,100.83,
3,Tutorial,34.09,112.58,
4,Midterm,28.12,110.0,
5,TakeHome,16.91,108.89,1
6,Final,28.06,108.89,3


In [7]:
CSV.validate("grades.csv")

┌ Warning: `CSV.validate` is deprecated. `CSV.read` now prints warnings on misshapen files.
│   caller = top-level scope at In[7]:1
└ @ Core In[7]:1


In [8]:
[cor(df[!, i], df[!, j]) for i in axes(df, 2), j in axes(df, 2)]

6×6 Array{Union{Missing, Float64},2}:
  1.0        0.0224759  0.431078  -0.0625435  missing  missing
  0.0224759  1.0        0.440115   0.215868   missing  missing
  0.431078   0.440115   1.0        0.135597   missing  missing
 -0.0625435  0.215868   0.135597   1.0        missing  missing
   missing    missing    missing    missing   missing  missing
   missing    missing    missing    missing   missing  missing

In [9]:
df2 = dropmissing(df);

In [10]:
describe(df2, :nmissing)

,variable,nmissing
,Symbol,Nothing
1,Prefix,
2,Assignment,
3,Tutorial,
4,Midterm,
5,TakeHome,
6,Final,


In [11]:
[cor(df2[!, i], df2[!, j]) for i in axes(df2, 2), j in axes(df2, 2)]

6×6 Array{Float64,2}:
  1.0        0.0484327  0.434525  -0.0586403  -0.0689997  0.0881758
  0.0484327  1.0        0.459001   0.200715    0.483206   0.286304 
  0.434525   0.459001   1.0        0.148637    0.238167   0.23987  
 -0.0586403  0.200715   0.148637   1.0         0.42719    0.724478 
 -0.0689997  0.483206   0.238167   0.42719     1.0        0.474231 
  0.0881758  0.286304   0.23987    0.724478    0.474231   1.0      

In [12]:
function cor2(x, y)
    df = dropmissing(DataFrame([x, y]))
    cor(df[!, 1], df[!, 2])
end

cor2 (generic function with 1 method)

In [13]:
[cor2(df[!, i], df[!, j]) for i in axes(df, 2), j in axes(df, 2)]

6×6 Array{Float64,2}:
  1.0        0.0224759  0.431078  -0.0625435  -0.0916684  0.0902548
  0.0224759  1.0        0.440115   0.215868    0.492297   0.291232 
  0.431078   0.440115   1.0        0.135597    0.209513   0.240551 
 -0.0625435  0.215868   0.135597   1.0         0.442408   0.725121 
 -0.0916684  0.492297   0.209513   0.442408    1.0        0.474231 
  0.0902548  0.291232   0.240551   0.725121    0.474231   1.0      

In [14]:
coalesce.([missing,2,missing,5],0)

4-element Array{Int64,1}:
 0
 2
 0
 5

In [15]:
coalesce(missing, missing, 2, 1)

2

In [16]:
completecases(df)

99-element BitArray{1}:
 1
 1
 1
 1
 1
 1
 1
 0
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [17]:
sin(missing)

missing

In [18]:
1 + missing

missing

In [19]:
s = ["a", "bb", missing, "dddd"]

4-element Array{Union{Missing, String},1}:
 "a"    
 "bb"   
 missing
 "dddd" 

In [20]:
(x -> isequal(x, missing) ? missing : length(x)).(s)

4-element Array{Union{Missing, Int64},1}:
 1       
 2       
  missing
 4       

In [21]:
1 == missing

missing

In [22]:
missing == missing

missing

In [23]:
1 < missing

missing

In [24]:
isequal(1, missing)

false

In [25]:
isequal(missing, missing)

true

In [26]:
isless(1, missing)

true

In [27]:
isless(Inf, missing)

true

In [28]:
using Statistics

abstract type CorMethod end
struct CorAll <: CorMethod end
struct CorComplete <: CorMethod end
struct CorPairwise <: CorMethod end

function Statistics.cor(df::DataFrame; method::CorMethod=CorAll())
    cor1(i, j) = nrow(df) == 0 ? missing : cor(df2[!, i], df2[!, j])

    function cor2(i, j)
        x = dropmissing(DataFrame([df2[!, i], df2[!, j]]))
        nrow(x) == 0 ? missing : cor(x[!, 1], x[!, 2])
    end

    use_cor = method == CorPairwise() ? cor2 : cor1
    df2 = method == CorComplete() ? dropmissing(df) : df

    m = Matrix{Union{Float64, Missing}}(undef, ncol(df), ncol(df))
    for i in 1:ncol(df), j in i:ncol(df)
        m[i, j] = use_cor(i, j)
        m[j, i] = m[i, j]
    end
    m
end

In [29]:
cor(df)

6×6 Array{Union{Missing, Float64},2}:
  1.0        0.0224759  0.431078  -0.0625435  missing  missing
  0.0224759  1.0        0.440115   0.215868   missing  missing
  0.431078   0.440115   1.0        0.135597   missing  missing
 -0.0625435  0.215868   0.135597   1.0        missing  missing
   missing    missing    missing    missing   missing  missing
   missing    missing    missing    missing   missing  missing

In [30]:
cor(df, method=CorComplete())

6×6 Array{Union{Missing, Float64},2}:
  1.0        0.0484327  0.434525  -0.0586403  -0.0689997  0.0881758
  0.0484327  1.0        0.459001   0.200715    0.483206   0.286304 
  0.434525   0.459001   1.0        0.148637    0.238167   0.23987  
 -0.0586403  0.200715   0.148637   1.0         0.42719    0.724478 
 -0.0689997  0.483206   0.238167   0.42719     1.0        0.474231 
  0.0881758  0.286304   0.23987    0.724478    0.474231   1.0      

In [31]:
cor(df, method=CorPairwise())

6×6 Array{Union{Missing, Float64},2}:
  1.0        0.0224759  0.431078  -0.0625435  -0.0916684  0.0902548
  0.0224759  1.0        0.440115   0.215868    0.492297   0.291232 
  0.431078   0.440115   1.0        0.135597    0.209513   0.240551 
 -0.0625435  0.215868   0.135597   1.0         0.442408   0.725121 
 -0.0916684  0.492297   0.209513   0.442408    1.0        0.474231 
  0.0902548  0.291232   0.240551   0.725121    0.474231   1.0      